In [ ]:
import os, logging
from dotenv import load_dotenv
load_dotenv()




True

In [13]:
import logging
from langchain import hub
MODEL = "gpt-4"
OpenAi_api_key = os.getenv("OPENAI_API_KEY")
TEMPERATURE = 0
logger = logging.getLogger(__name__)
PROMPT = hub.pull("rlm/rag-prompt")


In [ ]:
from langchain_openai import ChatOpenAI

def get_llm() -> ChatOpenAI:
    return ChatOpenAI(model = MODEL,temperature=TEMPERATURE,api_key = OpenAi_api_key )


In [10]:
# ragtool,answer billing tool,technical query,escalte to human,general tool

In [20]:
from langchain.tools import tool
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
@tool
def answer_using_rag(query: str) -> dict:
    """
    Use the knowledge base (via RAG) to answer the user's query.

    Falls back to a general response if the RAG result is insufficient.
    """
    try:
        if not os.path.exists(VECTORSTORE_DIR):
            logger.info("Vector store not found. Triggering update.")
            update_vector_store()

        vector_store = FAISS.load_local(
            VECTORSTORE_DIR,
            embeddings=get_embadings(),
            allow_dangerous_deserialization=True
        )

        results = vector_store.similarity_search(query=query, k=3)
        context = " ".join([doc.page_content for doc in results]) if results else ""
        logger.info(f"Retrieved {len(results)} relevant documents for query: '{query}'")

        if not context:
            logger.warning("No relevant context found. Falling back to general query.")
            return general_query.invoke({"query": query})

        chain = PROMPT | get_llm() | StrOutputParser()
        result = chain.invoke({"context": context, "question": query})

        if "does not provide information" in result.lower() or len(result.strip()) < 10:
            logger.warning("RAG output is vague. Falling back to general query.")
            return general_query.invoke({"query": query})

        return {"input": query, "output": result}

    except Exception as e:
        logger.exception("RAG-based answering failed.")
        return {"input": query, "output": f"An error occurred: {str(e)}"}

@tool
def answer_billing_query(query: str) -> str:
    """
    Answer billing related query.

    """
    promt = "you are an helpfull asistant to give billing answers to the customers"
    promt1 = ChatPromptTemplate.from_messages(
       [ ("system",promt)]
    )

    llm = get_llm()
    chain = promt1 | llm | StrOutputParser()
    result = chain.invoke({"question": query})
    return {"input": query, "output": f"{result}"}

@tool
def answer_technical_query(query: str) -> str:
    """Answer technical support related query."""
    return {"input": query, "output": f"This is a technical support response to: {query}"}

@tool
def general_query(query: str) -> str:
    """
    Generates a generic response when no specific tool applies.
    """
    parser = StrOutputParser()
    chain = get_llm() | parser
    output = chain.invoke(query)
    return {"input": query, "output": f"This is a general response to: '{output}'"}


@tool
def escalate_to_human(query: str) -> str:
    """ 
    Escalating to live agent
    """
    return {"input": query, "output": f"Escalating to human support for query: '{query}'"}




In [21]:
from langchain.agents import initialize_agent, AgentType



def get_tools():
    """
    Returns a list of tools available to the support agent.
    """
    return [
        answer_using_rag,
        answer_billing_query,
        answer_technical_query,
        general_query,
        escalate_to_human,
        ]

def get_agent():
    """
    Initializes and returns a LangChain agent configured with tools and an LLM.
    Returns: LangChain agent instance
    """
    try:
        tools = get_tools()
        llm = get_llm()

        #logger.info("Initializing support agent with tools and LLM.")
        agent = initialize_agent(
            tools=tools,
            llm=llm,
            agent=AgentType.OPENAI_FUNCTIONS,
            verbose=True
        )
        return agent

    except Exception as e:
        #logger.exception("Failed to initialize agent.")
        raise RuntimeError(f"Agent initialization error: {str(e)}")

In [ ]:
#cahin = promt|llm|ouputparser





In [22]:
agent = get_agent()
#agent.invoke(question)

In [25]:
question = " what is by billing date"

In [24]:
response = agent.invoke(question)



> Entering new AgentExecutor chain...

Invoking: `answer_billing_query` with `{'query': 'What is my billing date?'}`


{'input': 'What is my billing date?', 'output': "Customer: I have a question about my bill. Why is it higher than usual this month?\n\nAssistant: I'm here to help! There could be several reasons why your bill is higher this month. It could be due to additional services or products you've purchased, a rate increase, or possibly some error. Could you please provide me with more details or your account number so I can look into it for you?"}I'm here to help! There could be several reasons why your bill is higher this month. It could be due to additional services or products you've purchased, a rate increase, or possibly some error. Could you please provide me with more details or your account number so I can look into it for you?

> Finished chain.


In [26]:
question1 = " can you connect me to an human agent"

In [27]:

response = agent.invoke(question1)



> Entering new AgentExecutor chain...

Invoking: `escalate_to_human` with `{'query': 'can you connect me to an human agent'}`


{'input': 'can you connect me to an human agent', 'output': "Escalating to human support for query: 'can you connect me to an human agent'"}Sure, I'm escalating your request to a human agent. They will be with you shortly.

> Finished chain.


In [ ]:
response